# Comparing custom MODIS seasonal snow cover dataset vs automatic weather stations

In [1]:
import adlfs
import xarray as xr
import pathlib
import zarr
import rioxarray as rxr
import geopandas as gpd
import easysnowdata
import numpy as np
import matplotlib.pyplot as plt
import coiled

In [ ]:
cluster = coiled.Cluster(idle_timeout="10 minutes",
                        n_workers=20, 
                        worker_memory="16 GB", 
                        worker_cpu=8,
                        spot_policy="spot",
                        environ={"GDAL_DISABLE_READDIR_ON_OPEN": "EMPTY_DIR"},
                        workspace="uwtacolab",
                        )

client = cluster.get_client()

In [ ]:
client.restart()

In [2]:
sas_token = pathlib.Path("sas_token.txt").read_text()

mask_store = adlfs.AzureBlobFileSystem(
    account_name="snowmelt", credential=sas_token
).get_mapper("snowmelt/snow_cover/global_modis_snow_cover_4.zarr")

aggregated_mask_store = adlfs.AzureBlobFileSystem(
    account_name="snowmelt", credential=sas_token
).get_mapper("snowmelt/snow_cover/global_modis_snow_cover_4_aggregated.zarr")

In [ ]:
seasonal_snow_mask_ds = xr.open_zarr(mask_store,
                                     decode_coords='all',
                                     consolidated=True,
                                     mask_and_scale=True,
                                     )
seasonal_snow_mask_ds

In [ ]:
# seasonal_snow_mask_coarsened_ds = seasonal_snow_mask_ds.coarsen(x=10,y=10, boundary='trim').mean()
# seasonal_snow_mask_coarsened_ds

In [ ]:
def median_and_mad_with_min_obs(da, dim, min_count):
    count_mask = da.notnull().sum(dim=dim) >= min_count
    median = da.where(count_mask).median(dim=dim)
    abs_dev = np.abs(da - median)
    mad = abs_dev.where(count_mask).median(dim=dim)

    return median, mad

In [ ]:
# create an xarray dataset with the same data variables and same dimensions x and y, but don't include the dimension "water_year". 
# instead, there should be a different dimension named "statistic" and have coordinates "median" and "mad", with values assigned 
# based on the output of the median_and_mad_with_min_obs() function. you may have to loop through the data variables. 
# use the order: create the dataset skeleton first, then loop through the data variables to fill in the values.

def create_statistic_dataset(ds, min_count=5):
    statistic_ds = xr.Dataset(
        coords={
            "x": ds.x,
            "y": ds.y,
            "statistic": ["median", "mad"]
        }
    )  # Adjust chunk sizes as needed
    for var in ds.data_vars:
        median, mad = median_and_mad_with_min_obs(ds[var], dim="water_year", min_count=min_count)
        statistic_ds[var] = xr.concat([median, mad], dim="statistic").chunk({"x": 2400, "y": 2400, "statistic": 1})
        statistic_ds[var].attrs = ds[var].attrs  # Copy attributes from the original variable
    statistic_ds.attrs = ds.attrs  # Copy global attributes from the original dataset
    return statistic_ds

In [ ]:
seasonal_snow_mask_statistics_ds = create_statistic_dataset(seasonal_snow_mask_ds, min_count=3)
seasonal_snow_mask_statistics_ds

In [ ]:
seasonal_snow_mask_statistics_ds.to_zarr(
    aggregated_mask_store, mode="w", consolidated=True, write_empty_chunks=False,
)

In [ ]:
seasonal_snow_mask_statistics_read_ds = xr.open_zarr(
    aggregated_mask_store,
    decode_coords='all',
    consolidated=True,
    mask_and_scale=True,
)
seasonal_snow_mask_statistics_read_ds

In [ ]:
seasonal_snow_mask_statistics_coarsened_ds = (
    seasonal_snow_mask_statistics_read_ds.coarsen(x=20, y=20, boundary="trim")
    .mean()
    .compute()
)

seasonal_snow_mask_statistics_coarsened_ds

In [ ]:
f,ax=plt.subplots(figsize=(12,10))
seasonal_snow_mask_statistics_coarsened_ds['max_consec_snow_days'].sel(statistic='mad').plot.imshow(ax=ax, vmin=0, vmax=30, cmap='Reds')
ax.set_aspect('equal')

In [ ]:
f,ax=plt.subplots(figsize=(12,10))
seasonal_snow_mask_statistics_coarsened_ds['max_consec_snow_days'].sel(statistic='median').plot.imshow(ax=ax, vmin=0, vmax=365, cmap='viridis')
ax.set_aspect('equal')

In [ ]:
# v1 = 16
# v2 = 18
# h1 = 16
# h2 = 18

v1 = 2
v2 = 4
h1 = 9
h2 = 13

# v1 = 1
# v2 = 4
# h1 = 12
# h2 = 15

y_slice = slice(v1 * 2400, v2 * 2400)
x_slice = slice(h1 * 2400, h2 * 2400)

In [ ]:
# f,ax=plt.subplots(figsize=(12,10))
# seasonal_snow_mask_statistics_read_ds['max_consec_snow_days'].sel(statistic='mad').isel(y=y_slice,x=x_slice).plot.imshow(ax=ax, vmin=0, vmax=30, cmap='Reds')
# ax.set_aspect('equal')

In [ ]:
# f,ax=plt.subplots(figsize=(12,10))
# seasonal_snow_mask_statistics_read_ds['max_consec_snow_days'].sel(statistic='median').isel(y=y_slice,x=x_slice).plot.imshow(ax=ax, vmin=0, vmax=365, cmap='viridis')
# ax.set_aspect('equal')

In [ ]:
# f,ax=plt.subplots(figsize=(12,10))
# seasonal_snow_mask_statistics_read_ds['SDD_DOWY'].sel(statistic='median').isel(y=y_slice,x=x_slice).plot.imshow(ax=ax, vmin=0, vmax=365, cmap='viridis')
# ax.set_aspect('equal')

In [ ]:
diff_ds = seasonal_snow_mask_ds['SDD_DOWY'].isel(y=y_slice,x=x_slice)-seasonal_snow_mask_statistics_read_ds['SDD_DOWY'].isel(y=y_slice,x=x_slice).sel(statistic='median')
diff_ds

In [ ]:
diff_ds = diff_ds.compute()
diff_ds

In [ ]:
diff_ds.plot.imshow(col='water_year', col_wrap=5,cmap='RdBu',vmin=-40,vmax=40)

In [ ]:
# # add 10 year median and 10 year median absolute deviation for each variable "SAD_DOWY","SDD_DOWY","max_consec_snow_days" based on the previous cell

# min_count = 3  # Minimum number of observations required to compute median and MAD

# seasonal_snow_mask_coarsened_ds['SAD_DOWY_median'], seasonal_snow_mask_coarsened_ds['SAD_DOWY_mad'] = median_and_mad_with_min_obs(
#     seasonal_snow_mask_coarsened_ds['SAD_DOWY'], dim='water_year', min_count=min_count)
# seasonal_snow_mask_coarsened_ds['SDD_DOWY_median'], seasonal_snow_mask_coarsened_ds['SDD_DOWY_mad'] = median_and_mad_with_min_obs(
#     seasonal_snow_mask_coarsened_ds['SDD_DOWY'], dim='water_year', min_count=min_count)
# seasonal_snow_mask_coarsened_ds['max_consec_snow_days_median'], seasonal_snow_mask_coarsened_ds['max_consec_snow_days_mad'] = median_and_mad_with_min_obs(
#     seasonal_snow_mask_coarsened_ds['max_consec_snow_days'], dim='water_year', min_count=min_count)

# seasonal_snow_mask_coarsened_ds
